In [ ]:
import polars as pl
pl.Config.set_tbl_rows(50)  # Больше строк в таблице
pl.Config.set_tbl_width_chars(150)  # Ширина вывода

polars.config.Config

In [46]:
# Загрузка данных
df = pl.read_csv("train.csv")

print(f"Размер датасета: {df.shape}")
print(f"Колонки: {df.columns}")
print()



Размер датасета: (891, 12)
Колонки: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']



In [47]:
# 2. Основная информация о датасете
print("Основная информация о датасете")
print("Типы данных:")
print(dict(df.schema))
print()

print("Число пропусков:")
null_counts = df.null_count()
print(null_counts)
print()

print("Основные статистики (ключевые числовые колонки):")
numeric_stats = df.select(["Age", "Fare", "SibSp", "Parch", "Pclass", "Survived"]).describe()
print(numeric_stats)
print()



Основная информация о датасете
Типы данных:
{'PassengerId': Int64, 'Survived': Int64, 'Pclass': Int64, 'Name': String, 'Sex': String, 'Age': Float64, 'SibSp': Int64, 'Parch': Int64, 'Ticket': String, 'Fare': Float64, 'Cabin': String, 'Embarked': String}

Число пропусков:
shape: (1, 12)
┌─────────────┬──────────┬────────┬──────┬───┬────────┬──────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name ┆ … ┆ Ticket ┆ Fare ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---  ┆   ┆ ---    ┆ ---  ┆ ---   ┆ ---      │
│ u32         ┆ u32      ┆ u32    ┆ u32  ┆   ┆ u32    ┆ u32  ┆ u32   ┆ u32      │
╞═════════════╪══════════╪════════╪══════╪═══╪════════╪══════╪═══════╪══════════╡
│ 0           ┆ 0        ┆ 0      ┆ 0    ┆ … ┆ 0      ┆ 0    ┆ 687   ┆ 2        │
└─────────────┴──────────┴────────┴──────┴───┴────────┴──────┴───────┴──────────┘

Основные статистики (ключевые числовые колонки):
shape: (9, 7)
┌────────────┬───────────┬───────────┬──────────┬──────────┬──────────┬────

In [50]:
# 3. Количество пассажиров каждого класса
print("Количество пассажиров каждого класса")
pclass_counts = df.group_by("Pclass").agg(
    pl.len().alias("count")
).sort("Pclass")
print(pclass_counts)
print()

Количество пассажиров каждого класса
shape: (3, 2)
┌────────┬───────┐
│ Pclass ┆ count │
│ ---    ┆ ---   │
│ i64    ┆ u32   │
╞════════╪═══════╡
│ 1      ┆ 216   │
│ 2      ┆ 184   │
│ 3      ┆ 491   │
└────────┴───────┘



In [49]:
# 4. Количество выживших мужчин и женщин
print("Количество выживших мужчин и женщин")
survived_by_sex = df.filter(pl.col("Survived") == 1).group_by("Sex").agg(
    pl.len().alias("survived_count")
)
print(survived_by_sex)
print()

Количество выживших мужчин и женщин
shape: (2, 2)
┌────────┬────────────────┐
│ Sex    ┆ survived_count │
│ ---    ┆ ---            │
│ str    ┆ u32            │
╞════════╪════════════════╡
│ male   ┆ 109            │
│ female ┆ 233            │
└────────┴────────────────┘



In [51]:
# 5. Пассажиры старше 44 лет
print("Пассажиры старше 44 лет")
older_passengers = df.filter(pl.col("Age") > 44)
print(f"Количество пассажиров старше 44 лет: {len(older_passengers)}")
print()

print("Первые 10 пассажиров старше 44 лет:")
print(older_passengers.head(10).select(["PassengerId", "Name", "Age", "Sex", "Pclass", "Survived"]))

Пассажиры старше 44 лет
Количество пассажиров старше 44 лет: 115

Первые 10 пассажиров старше 44 лет:
shape: (10, 6)
┌─────────────┬─────────────────────────────────┬──────┬────────┬────────┬──────────┐
│ PassengerId ┆ Name                            ┆ Age  ┆ Sex    ┆ Pclass ┆ Survived │
│ ---         ┆ ---                             ┆ ---  ┆ ---    ┆ ---    ┆ ---      │
│ i64         ┆ str                             ┆ f64  ┆ str    ┆ i64    ┆ i64      │
╞═════════════╪═════════════════════════════════╪══════╪════════╪════════╪══════════╡
│ 7           ┆ McCarthy, Mr. Timothy J         ┆ 54.0 ┆ male   ┆ 1      ┆ 0        │
│ 12          ┆ Bonnell, Miss. Elizabeth        ┆ 58.0 ┆ female ┆ 1      ┆ 1        │
│ 16          ┆ Hewlett, Mrs. (Mary D Kingcome… ┆ 55.0 ┆ female ┆ 2      ┆ 1        │
│ 34          ┆ Wheadon, Mr. Edward H           ┆ 66.0 ┆ male   ┆ 2      ┆ 0        │
│ 53          ┆ Harper, Mrs. Henry Sleeper (My… ┆ 49.0 ┆ female ┆ 1      ┆ 1        │
│ 55          ┆ Ostby, 